this notebook is used to study velocity dispersion maps and velocity dispersion distribution

In [ ]:
import time
from multiprocessing import Pool
from postprocessing import *

        
def weighted_std(values, weights=None):
    """
    Return the weighted standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return np.sqrt(variance)

def sigma(x):
    """
    Calculate velocity dispersion for x and y_grid (global parameter...)
    """
    result = []
    temperatures = get_temp(output_directory + filename, 5/3)
    for y in y_grid[:-1]:
        mask = ((snap_data['PartType0/Coordinates'][:][:,2] > x) & 
                (snap_data['PartType0/Coordinates'][:][:,2] < x + dx) &
                (snap_data['PartType0/Coordinates'][:][:,1] > y) &
                (snap_data['PartType0/Coordinates'][:][:,1] < y + dy) &
                (temperatures < 1e5))
        try: 
            velocity_dispersion = weighted_std(snap_data['PartType0/Velocities'][:][:,0][mask], weights=masses)
        except: 
            velocity_dispersion = np.nan
        result.append([x, y, velocity_dispersion])
    return result

In [ ]:
mpl.rcParams['figure.dpi']= 200

## observational resolution

In [ ]:
0.044 * 300000/70 * np.sin(np.deg2rad(0.5 /60 /60)) * 1e6 # in pc resolution from observations?!

In [ ]:
N = 100
x_grid = np.linspace(0, 1000, N + 1)
y_grid = np.linspace(0, 1000, N + 1)
dx, dy = np.diff(x_grid)[0], np.diff(y_grid)[0]

In [ ]:
density = '300'
mach    = '8'
jet     = '41'
stage   = 'early'

In [ ]:
#simulation_directory = f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/jet{jet}_{stage}'
simulation_directory = f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/turb'

output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"\

In [ ]:
i_file = 0
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory + filename, "r")

## high res maps

In [ ]:
starttime = time.time()
pool = Pool(32)
sigma_result = pool.map(sigma, x_grid[:-1])
pool.close()
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

In [ ]:
sigma_result = np.array(sigma_result)

In [ ]:
sigma_result = sigma_result.reshape(N * N, 3)

In [ ]:
sigma_result = sigma_result.T

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
c = ax.scatter(sigma_result[0], sigma_result[1], c=sigma_result[2], cmap='plasma', norm=colors.LogNorm(vmin=50, vmax=200), s=4, marker='s')
ax.set_xlabel('z [pc]')
ax.set_ylabel('y [pc]')
ax.set_aspect('1')
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)
ax.scatter(500, 500, c='white', s=10)
plt.colorbar(c, ax=ax, label=r'$\sigma_x$ [km / s]')
# plt.savefig(figures_directory + f'veldispersion_{i_file}.png', dpi=300, bbox_inches='tight')

In [ ]:
# from PIL import Image
# import glob
# from natsort import natsorted

# # make gif
# #--------------------------
# def crop_img(im):
#     width, height = im.size
#     left = 9
#     top =  3
#     right = width - 3
#     bottom = height - 9
#     im = im.crop((left, top, right, bottom))
#     return im

# ifilename = figures_directory + '/veldispersion*.png'
# ofilename = figures_directory + '/veldispersion.gif'
# imgs = natsorted(glob.glob(ifilename))

# timestep=4

# frames = []
# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(crop_img(new_frame))

# frames[0].save(ofilename, format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=len(imgs) * timestep, loop=0)

## velocity distribution per pixel:

In [ ]:
x = x_grid[N // 4]
y = y_grid[N // 4]

In [ ]:
result = []
temperatures = get_temp(output_directory + filename, 5/3)

mask = ((snap_data['PartType0/Coordinates'][:][:,0] > x) & 
        (snap_data['PartType0/Coordinates'][:][:,0] < x + dx) &
        (snap_data['PartType0/Coordinates'][:][:,1] > y) &
        (snap_data['PartType0/Coordinates'][:][:,1] < y + dy) &
        (snap_data['PartType0/Coordinates'][:][:,2] > 0) &
        (temperatures < 10 ** 4))

velocities = snap_data['PartType0/Velocities'][:][:,2][mask]
masses     = snap_data['PartType0/Masses'][:][mask]

In [ ]:
mean = np.mean(velocities)
std = weighted_std(velocities, weights=masses)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 4))
c = ax[0].scatter(sigma_result[0], sigma_result[1], c=sigma_result[2], cmap='plasma', norm=colors.LogNorm(vmin=50, vmax=200), s=2.5, marker='s')
ax[0].set_xlabel('x [pc]')
ax[0].set_ylabel('y [pc]')
ax[0].set_aspect('1')
ax[0].set_xlim(0, 1000)
ax[0].set_ylim(0, 1000)
ax[0].scatter(500, 500, c='white', s=10, marker='o')
ax[0].scatter(x, y, c='white', marker='s', s=10)
cb = plt.colorbar(c, ax=ax[0])
cb.set_label(label=r'$\sigma_z$ [km / s]', labelpad=-10)

ax[1].hist(velocities, bins=30, weights=masses)
ax[1].set_xlabel('$v_z$ [km / s]')
ax[1].set_ylabel('number')
ax[1].set_xlim(-300, 300)
ax[1].errorbar(mean, 5000, xerr=std, ms=5, capsize=5, fmt='o', color='black')
plt.subplots_adjust(wspace=0.3)

### dispersion in the whole box as a function of time

In [ ]:
stds = []
bins = np.linspace(-300, 300, 51)

In [ ]:
i_file =  -1

while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    
    temperatures = get_temp(output_directory + filename, 5/3)
    mask = (temperatures < 10 ** 4.5)
    
    velocities = snap_data['PartType0/Velocities'][:][:,2][mask]
    masses     = snap_data['PartType0/Masses'][:][mask]

    std = weighted_std(velocities, weights=masses)
    
    if i_file==0 or i_file==7:
        plt.hist(velocities, weights=masses, bins=bins, alpha=0.5, label=f't={np.round(get_time_from_snap(snap_data), 2)} Myr')
        plt.xlabel(r'$v_z$ [km / s]')
        plt.ylabel('mass weighted cells number')
        plt.xlim(-300, 300)
        plt.ylim(1, 7e8)
        plt.legend()
    stds.append([get_time_from_snap(snap_data), std])

In [ ]:
stds = np.array(stds).T

In [ ]:
plt.plot(stds[0], stds[1])
plt.ylim(60, 100)
plt.xlabel('time [Myr]')
plt.ylabel(r'$\sigma_z$ [km / s]')